### 01. Connect to your Google Drive

Run the code below to connect to your Google Drive folders.
You will be prompted for permissions to access your drive.


In [ ]:
try:
    from google.colab import drive

    drive.mount("/content/drive", force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False


### 02. Verify if GPUs are turned on and available

If the `nvidia-smi` script does not return any GPUs, you need to turn on GPUs in:

```
Menu -> Runtime -> Change runtime type -> Hardware Accelerator Dropdown -> select "GPU"
```

In [ ]:
# Verify if GPUs are available
!nvidia-smi

In [ ]:
# Install dependencies and clone StyleGAN3

!pip install torch==1.9.0 torchvision==0.10.0
!git clone https://github.com/NVlabs/stylegan3.git
!pip install ninja

### 03. Create folders for training

Create a folder structure in your Google Drive:

```
├── stylegan
│   ├── images
│   │   ├── <Your dataset folder containing images with dimensions of power of 2 (ex. - 64x64, 128x128, 1024x1024)>
│   ├── datasets
│   ├── experiments
```

You can use [ImageMagick's](https://imagemagick.org/) batch image processor script `mogrify` locally before uploading to Google Drive to make sure all your images

Take note of your raw dataset folder name and put it in the `RAW_DATASET_FOLDER_NAME` variable below.


In [ ]:
import os

# Change this to your dataset folder name
RAW_DATASET_FOLDER_NAME = "<CHANGE_ME>" 

STYLEGAN_ROOT_FOLDER = os.path.join(
    os.sep, "content", "drive", "MyDrive", "stylegan"
)
RAW_DATASET_PATH = os.path.join(STYLEGAN_ROOT_FOLDER, "images", RAW_DATASET_FOLDER_NAME)
ZIP_DATASET_PATH = os.path.join(STYLEGAN_ROOT_FOLDER, "datasets", RAW_DATASET_FOLDER_NAME + ".zip")
EXPERIMENTS_PATH = os.path.join(STYLEGAN_ROOT_FOLDER, "experiments")


In [ ]:
dataset_creation_cmd = f"python /content/stylegan3/dataset_tool.py --source={RAW_DATASET_PATH} --dest={ZIP_DATASET_PATH}"
!{dataset_creation_cmd}

In [ ]:
# Verifies your dataset for inconsistent sizes and colour formats
# Script created by @jeffheaton

from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm


files = [f for f in listdir(RAW_DATASET_PATH) if isfile(join(RAW_DATASET_PATH, f))]

base_size = None
for file in tqdm(files):
    file2 = os.path.join(RAW_DATASET_PATH, file)
    img = Image.open(file2)
    sz = img.size
    if base_size and sz != base_size:
        print(f"Inconsistant size: {file2}")
    elif img.mode != "RGB":
        print(f"Inconsistant color format: {file2}")
    else:
        base_size = sz


### 04. Training

You can play around with different configurations here. A few descriptions:

- `CFG` - you can pick `stylegan3-t` (only designed for translation equivariance) or `stylegan3-r` (features high-quality, though not visually perfect rotation equivariance). More details in the [official NVIDIA StyleGAN3 documentation](https://nvlabs.github.io/stylegan3/)
- `BATCH_SIZE` - the number of samples that will be passed through to the network at one time
- `GAMMA` - R1 regularization weight
- `MIRROR` - `1` to augment the dataset with random x-flips, `0` otherwise
- `SNAP` - number of epochs before saving a new checkpoint in the `experiments` folder on Google Drive (for Google Colab Free, it is recommended to keep this low, say `5` or `10`)

You can refer to the [official config documentation](https://github.com/NVlabs/stylegan3/blob/main/docs/configs.md) to see more config parameters.

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/research/data/gan/experiments"
DATA = "/content/drive/MyDrive/research/data/gan/dataset/anime_faces.zip"
CFG = "stylegan3-t"
BATCH_SIZE = 32
GAMMA = 1
MIRROR = 1
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan3/train.py --snap={SNAP} --outdir={EXPERIMENTS_PATH} --data={ZIP_DATASET_PATH} --cfg={CFG} --gpus=1 --batch={BATCH_SIZE} --gamma={GAMMA} --mirror={MIRROR}"
!{cmd}

### 05. Resuming training

Your Colab instance will automatically be turned off after 12 hours (Colab Free) or 24 hours (Colab Pro/Pro+).
You can resume training with the code below.

Check your Google Drive folder to check the `CHECKPOINT_FOLDER` and `NETWORK` name in the `experiments` folder.
An example of this is:

```
├── stylegan
│   ├── images
│   ├── dataset
│   ├── experiments
│   │   ├── 00002-dogs_images-auto1-resumecustom --> This is the `CHECKPOINT_FOLDER`
│   │   │   ├── network-snapshot-000160.pkl --> This is the `NETWORK`
```

You can use the `NETWORK` with the latest epoch to continue.
The confgurations are the same as the ones in *04. Training*.

In [ ]:
# Resume Training

import os

# Modify these to suit your needs
CHECKPOINT_FOLDER = "<CHANGE_ME>"
NETWORK = "<CHANGE_ME>"

RESUME = os.path.join(EXPERIMENTS_PATH, CHECKPOINT_FOLDER, NETWORK)
CFG = "stylegan3-t"
BATCH_SIZE = 32
GAMMA = 1
MIRROR = 1
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan3/train.py --resume={RESUME} --snap={SNAP} --outdir={EXPERIMENTS_PATH} --data={ZIP_DATASET_PATH} --cfg={CFG} --gpus=1 --batch={BATCH_SIZE} --gamma={GAMMA} --mirror={MIRROR}"
!{cmd}